In [1]:
cd("$(homedir())/OneDrive/Documents/GitHub/enso_project.jl")
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\Andrea\OneDrive\Documents\GitHub\enso_project.jl`


In [6]:
using CSV, DataFrames, enso_project, NODEData

┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   Base.PkgId(Base.UUID("e3ecd195-ca82-5397-9546-f380c1e34951"), "NonlinearSolveBaseSparseMatrixColoringsExt")
│   Base.PkgId(Base.UUID("385e4588-a1a0-5c1d-98fa-d45bf6f8ecf9"), "LinearSolveKernelAbstractionsExt")
│   Base.PkgId(Base.UUID("3bcf3b12-2128-5d18-8b3b-bcdd6f83637b"), "WeightInitializersGPUArraysExt")
│   Base.PkgId(Base.UUID("b00db79b-61e3-50fb-b26f-2d35b2d9e4ed"), "DiffEqBaseChainRulesCoreExt")
│   Base.PkgId(Base.UUID("8913a72c-1f9b-4ce2-8d82-65094dcecaec"), "NonlinearSolve")
│   Base.PkgId(Base.UUID("7edab7de-1038-5e4f-97a7-6bfc75d44324"), "NonlinearSolveQuasiNewtonForwardDiffExt")
│   Base.PkgId(Base.UUID("1dea7af3-3e70-54e6-95c3-0bf5283fa5ed"), "OrdinaryDiffEq")
│   Base.PkgId(Base.UUID("0d7ed370-da01-4f52-bd93-41d350b8b718"), "StaticArrayInterface")
│   Base.PkgId(Base.UUID("63d416d0-6995-5965-81e0-55251226d976"), "NonlinearSolveBaseForwardDiffExt")
│   Base.PkgId(Base.UUID("63c18a36-062a-441

### Data

In [7]:
# read training data and convert to appropriate format
df = CSV.read("data/train_sst_34_anomaly_embedded_15_04.txt", DataFrame; delim=',', ignorerepeated=true, header=0)
df = df[2:end,:]
df = permutedims(df)
df_train = parse.(Float32,df)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,⋯
1,0.08,-0.2,-0.14,0.02,0.49,0.65,0.27,0.86,1.24,1.73,1.68,2.21,2.13,1.81,1.22,0.68,0.68,0.45,-0.33,-0.44,-0.38,-1.02,-1.29,-1.16,-0.96,-0.16,-0.52,-0.54,-0.69,-1.03,-0.68,-0.54,-0.41,-0.75,-1.3,-1.48,-1.3,-0.92,-0.89,-1.1,-1.08,-1.08,-0.8,-0.63,-0.77,-0.74,-0.59,-0.59,-0.72,-0.78,-0.6,-0.52,-0.56,-0.33,-0.04,0.19,0.4,0.51,0.54,0.82,0.94,0.82,0.89,0.45,0.32,0.72,1.04,1.2,1.32,0.97,0.85,0.77,0.67,0.22,0.06,-0.55,-1.32,-1.7,-1.73,-1.43,-0.93,-2.01,-2.22,-2.01,-2.0,-1.48,-1.44,-1.32,-0.89,-0.67,-0.57,-0.72,-0.51,-0.55,-0.62,-0.2,-0.11,0.01,-0.18,-0.06,⋯
2,0.27,0.86,1.24,1.73,1.68,2.21,2.13,1.81,1.22,0.68,0.68,0.45,-0.33,-0.44,-0.38,-1.02,-1.29,-1.16,-0.96,-0.16,-0.52,-0.54,-0.69,-1.03,-0.68,-0.54,-0.41,-0.75,-1.3,-1.48,-1.3,-0.92,-0.89,-1.1,-1.08,-1.08,-0.8,-0.63,-0.77,-0.74,-0.59,-0.59,-0.72,-0.78,-0.6,-0.52,-0.56,-0.33,-0.04,0.19,0.4,0.51,0.54,0.82,0.94,0.82,0.89,0.45,0.32,0.72,1.04,1.2,1.32,0.97,0.85,0.77,0.67,0.22,0.06,-0.55,-1.32,-1.7,-1.73,-1.43,-0.93,-2.01,-2.22,-2.01,-2.0,-1.48,-1.44,-1.32,-0.89,-0.67,-0.57,-0.72,-0.51,-0.55,-0.62,-0.2,-0.11,0.01,-0.18,-0.06,0.04,-0.26,-0.03,0.09,-0.03,0.18,⋯
3,2.13,1.81,1.22,0.68,0.68,0.45,-0.33,-0.44,-0.38,-1.02,-1.29,-1.16,-0.96,-0.16,-0.52,-0.54,-0.69,-1.03,-0.68,-0.54,-0.41,-0.75,-1.3,-1.48,-1.3,-0.92,-0.89,-1.1,-1.08,-1.08,-0.8,-0.63,-0.77,-0.74,-0.59,-0.59,-0.72,-0.78,-0.6,-0.52,-0.56,-0.33,-0.04,0.19,0.4,0.51,0.54,0.82,0.94,0.82,0.89,0.45,0.32,0.72,1.04,1.2,1.32,0.97,0.85,0.77,0.67,0.22,0.06,-0.55,-1.32,-1.7,-1.73,-1.43,-0.93,-2.01,-2.22,-2.01,-2.0,-1.48,-1.44,-1.32,-0.89,-0.67,-0.57,-0.72,-0.51,-0.55,-0.62,-0.2,-0.11,0.01,-0.18,-0.06,0.04,-0.26,-0.03,0.09,-0.03,0.18,-0.09,0.22,0.33,0.14,-0.08,0.08,⋯
4,-0.33,-0.44,-0.38,-1.02,-1.29,-1.16,-0.96,-0.16,-0.52,-0.54,-0.69,-1.03,-0.68,-0.54,-0.41,-0.75,-1.3,-1.48,-1.3,-0.92,-0.89,-1.1,-1.08,-1.08,-0.8,-0.63,-0.77,-0.74,-0.59,-0.59,-0.72,-0.78,-0.6,-0.52,-0.56,-0.33,-0.04,0.19,0.4,0.51,0.54,0.82,0.94,0.82,0.89,0.45,0.32,0.72,1.04,1.2,1.32,0.97,0.85,0.77,0.67,0.22,0.06,-0.55,-1.32,-1.7,-1.73,-1.43,-0.93,-2.01,-2.22,-2.01,-2.0,-1.48,-1.44,-1.32,-0.89,-0.67,-0.57,-0.72,-0.51,-0.55,-0.62,-0.2,-0.11,0.01,-0.18,-0.06,0.04,-0.26,-0.03,0.09,-0.03,0.18,-0.09,0.22,0.33,0.14,-0.08,0.08,0.25,0.39,0.47,0.6,0.31,0.64,⋯
5,-0.96,-0.16,-0.52,-0.54,-0.69,-1.03,-0.68,-0.54,-0.41,-0.75,-1.3,-1.48,-1.3,-0.92,-0.89,-1.1,-1.08,-1.08,-0.8,-0.63,-0.77,-0.74,-0.59,-0.59,-0.72,-0.78,-0.6,-0.52,-0.56,-0.33,-0.04,0.19,0.4,0.51,0.54,0.82,0.94,0.82,0.89,0.45,0.32,0.72,1.04,1.2,1.32,0.97,0.85,0.77,0.67,0.22,0.06,-0.55,-1.32,-1.7,-1.73,-1.43,-0.93,-2.01,-2.22,-2.01,-2.0,-1.48,-1.44,-1.32,-0.89,-0.67,-0.57,-0.72,-0.51,-0.55,-0.62,-0.2,-0.11,0.01,-0.18,-0.06,0.04,-0.26,-0.03,0.09,-0.03,0.18,-0.09,0.22,0.33,0.14,-0.08,0.08,0.25,0.39,0.47,0.6,0.31,0.64,0.89,1.5,1.67,1.55,1.16,1.0,⋯


In [8]:
# read validation data and convert to appropriate format
df = CSV.read("data/val_sst_34_anomaly_embedded_15_04.txt", DataFrame; delim=',', ignorerepeated=true, header=0)
df = df[2:end,:]
df = permutedims(df)
df_val = parse.(Float32,df)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32
1,0.82,0.83,1.02,1.26,1.65,1.79,2.21,2.72,2.39,2.47,2.23,1.61,0.98,0.27,0.02,-0.42,-0.49,-0.43,-0.7,-0.64,-0.33,-0.26,0.1,0.21,0.36,0.42,0.39,0.28,-0.06,-0.32,-0.37,-0.91,-0.93,-0.7,-0.67,-0.73,-0.49,-0.11,0.15,0.22,0.28,0.33,0.86,0.86,0.9,0.67,0.71,0.97,0.72,0.75,0.6,0.42,0.15,-0.01,0.52,0.46,0.44,0.52,0.35,0.46,0.41,-0.21,-0.27,-0.14,-0.52,-0.64,-1.13,-1.23,-0.99,-0.9,-0.87,-0.55,-0.57,-0.32
2,2.21,2.72,2.39,2.47,2.23,1.61,0.98,0.27,0.02,-0.42,-0.49,-0.43,-0.7,-0.64,-0.33,-0.26,0.1,0.21,0.36,0.42,0.39,0.28,-0.06,-0.32,-0.37,-0.91,-0.93,-0.7,-0.67,-0.73,-0.49,-0.11,0.15,0.22,0.28,0.33,0.86,0.86,0.9,0.67,0.71,0.97,0.72,0.75,0.6,0.42,0.15,-0.01,0.52,0.46,0.44,0.52,0.35,0.46,0.41,-0.21,-0.27,-0.14,-0.52,-0.64,-1.13,-1.23,-0.99,-0.9,-0.87,-0.55,-0.57,-0.32,-0.04,-0.19,-0.28,-0.27,-0.77,-0.85
3,0.98,0.27,0.02,-0.42,-0.49,-0.43,-0.7,-0.64,-0.33,-0.26,0.1,0.21,0.36,0.42,0.39,0.28,-0.06,-0.32,-0.37,-0.91,-0.93,-0.7,-0.67,-0.73,-0.49,-0.11,0.15,0.22,0.28,0.33,0.86,0.86,0.9,0.67,0.71,0.97,0.72,0.75,0.6,0.42,0.15,-0.01,0.52,0.46,0.44,0.52,0.35,0.46,0.41,-0.21,-0.27,-0.14,-0.52,-0.64,-1.13,-1.23,-0.99,-0.9,-0.87,-0.55,-0.57,-0.32,-0.04,-0.19,-0.28,-0.27,-0.77,-0.85,-1.05,-0.83,-0.71,-0.89,-0.95,-1.05
4,-0.7,-0.64,-0.33,-0.26,0.1,0.21,0.36,0.42,0.39,0.28,-0.06,-0.32,-0.37,-0.91,-0.93,-0.7,-0.67,-0.73,-0.49,-0.11,0.15,0.22,0.28,0.33,0.86,0.86,0.9,0.67,0.71,0.97,0.72,0.75,0.6,0.42,0.15,-0.01,0.52,0.46,0.44,0.52,0.35,0.46,0.41,-0.21,-0.27,-0.14,-0.52,-0.64,-1.13,-1.23,-0.99,-0.9,-0.87,-0.55,-0.57,-0.32,-0.04,-0.19,-0.28,-0.27,-0.77,-0.85,-1.05,-0.83,-0.71,-0.89,-0.95,-1.05,-0.68,-0.62,-0.98,-0.93,-0.85,-0.93
5,0.36,0.42,0.39,0.28,-0.06,-0.32,-0.37,-0.91,-0.93,-0.7,-0.67,-0.73,-0.49,-0.11,0.15,0.22,0.28,0.33,0.86,0.86,0.9,0.67,0.71,0.97,0.72,0.75,0.6,0.42,0.15,-0.01,0.52,0.46,0.44,0.52,0.35,0.46,0.41,-0.21,-0.27,-0.14,-0.52,-0.64,-1.13,-1.23,-0.99,-0.9,-0.87,-0.55,-0.57,-0.32,-0.04,-0.19,-0.28,-0.27,-0.77,-0.85,-1.05,-0.83,-0.71,-0.89,-0.95,-1.05,-0.68,-0.62,-0.98,-0.93,-0.85,-0.93,-0.84,-0.69,-0.44,-0.01,0.19,0.47


In [9]:
# read test data and convert to appropriate format
df = CSV.read("data/test_sst_34_anomaly_embedded_15_04.txt", DataFrame; delim=',', ignorerepeated=true, header=0)
df = df[2:end,:]
df = permutedims(df)
df_test = parse.(Float32,df)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32
1,-0.04,-0.19,-0.28,-0.27,-0.77,-0.85,-1.05,-0.83,-0.71,-0.89,-0.95,-1.05,-0.68,-0.62,-0.98,-0.93,-0.85,-0.93,-0.84,-0.69
2,-1.05,-0.83,-0.71,-0.89,-0.95,-1.05,-0.68,-0.62,-0.98,-0.93,-0.85,-0.93,-0.84,-0.69,-0.44,-0.01,0.19,0.47,0.88,1.07
3,-0.68,-0.62,-0.98,-0.93,-0.85,-0.93,-0.84,-0.69,-0.44,-0.01,0.19,0.47,0.88,1.07,1.3,1.53,1.59,1.9,1.99,1.78
4,-0.84,-0.69,-0.44,-0.01,0.19,0.47,0.88,1.07,1.3,1.53,1.59,1.9,1.99,1.78,1.53,1.24,0.81,0.31,0.24,0.21
5,0.88,1.07,1.3,1.53,1.59,1.9,1.99,1.78,1.53,1.24,0.81,0.31,0.24,0.21,-0.07,-0.15,-0.28,-0.14,-0.62,-0.71


In [10]:
t_train = Float32.(0:size(df_train,2)-1)
data_train = Array(df_train)

t_valid = Float32.(0:size(df_val,2)-1)
data_valid = Array(df_val)

t_test = Float32.(0:size(df_test,2)-1)
data_test = Array(df_test)

train = NODEDataloader(Float32.(data_train), Float32.(t_train), 2)
valid = NODEDataloader(Float32.(data_valid), Float32.(t_valid), 2)
test = NODEDataloader(Float32.(data_test), Float32.(t_test), 2)

NODEData{Matrix{Float32},Int64} with 19 batches with length 2

### Hyperparameter tuning

In [11]:
sampler = enso_project.RandomSampler(N_weights=10:30, N_hidden_layers=1:4, activation=["relu","swish"], τ_max=2:4, eta=[1f-2,1f-3,1f-4], 
eta_decrease=[5,10,15])
N_samples = 15
N_epochs = 20
seeds = 111:111:2220
#opt_loss, opt_hpars = enso_project.train_and_validate_node(train, valid, N_samples, N_epochs, seeds, sampler)

111:111:2220

### Prediction

Set up optimal model

In [ ]:
Random.seed!(seeds[opt_loss[2]])

# set optimal hyperparameters
N_weights = opt_hpars[:N_weights]
N_hidden_layers = opt_hpars[:N_hidden_layers]
act_func = opt_hpars[:activation]
if act_func == "relu"
    activation = relu 
else
    activation = swish
end
# create network structure of optimal hyperparameters
hidden_layers = [Flux.Dense(N_weights, N_weights, activation) for i=1:N_hidden_layers]
nn = Chain(Flux.Dense(5, N_weights, activation), hidden_layers...,  Flux.Dense(N_weights, 5)) 
p, re_nn = Flux.destructure(nn)

# set optimal parameters
p = opt_ps[opt_loss[2]]

# define optimal NDE model
u0 = Vector(train.data[:,1])
dt = train.t[2] - train.t[1]
neural_ode(u, p, t) = re_nn(p)(u)
basic_tgrad(u,p,t) = zero(u)
odefunc = ODEFunction{false}(neural_ode,tgrad=basic_tgrad)
node_prob = ODEProblem(odefunc, u0, (Float32(0.),Float32(dt)), p)
model = enso_project.ChaoticNDE(node_prob)

enso_project.ChaoticNDE{Vector{Float32}, ODEProblem{Vector{Float32}, Tuple{Float32, Float32}, false, Vector{Float32}, ODEFunction{false, SciMLBase.FullSpecialize, typeof(neural_ode), LinearAlgebra.UniformScaling{Bool}, Nothing, typeof(basic_tgrad), Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing, Nothing, Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, SciMLBase.StandardODEProblem}, Tsit5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}, Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}}(Float32[-0.71703404, -0.049192842, 0.24385516, -0.40062472, -0.5530653, 0.36369967, 0.076058105, 0.34847203, 0.15863287, 0.42809996  …  0.0015036415, -0.013067125, -0.001047409, -0.00036992814, -0.002136838, 0.012067066, -0.016757477, -0.017651776, -0.014470856, 0.018294733], ODEProblem{Vector{Float32}, Tuple{Float32, Float32}, false, Vector{Float

Check optimal model's forecasting performance on the testing set

In [ ]:
enso_project.plot_node(model, test)